In [ ]:
# !pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import joblib
import pickle
import json

# ml frameworks
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    precision_recall_curve,
    auc,
    RocCurveDisplay,
    PrecisionRecallDisplay,
)

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
#time management
from tqdm import tqdm

MultiClassification

In [ ]:
X_train_csv = pd.read_csv("/content/drive/MyDrive/MC-II/CARVEME/50/X_train_MC-II-50.csv")
X_val_csv = pd.read_csv("/content/drive/MyDrive/MC-II/CARVEME/50/X_val_MC-II-50.csv")
X_test_csv = pd.read_csv("/content/drive/MyDrive/MC-II/CARVEME/50/X_test_MC-II-50.csv")
y_train_csv = pd.read_csv("/content/drive/MyDrive/MC-II/CARVEME/50/y_train_MC-II-50.csv")
y_val_csv = pd.read_csv("/content/drive/MyDrive/MC-II/CARVEME/50/y_val_MC-II-50.csv")
y_test_csv = pd.read_csv("/content/drive/MyDrive/MC-II/CARVEME/50/y_test_MC-II-50.csv")

In [ ]:
def to_npy(X_train_csv, X_val_csv, X_test_csv, y_train_csv, y_val_csv, y_test_csv):
    X_train = X_train_csv.to_numpy()
    X_val = X_val_csv.to_numpy()
    X_test = X_test_csv.to_numpy()
    y_train = y_train_csv.to_numpy()
    y_val = y_val_csv.to_numpy()
    y_test = y_test_csv.to_numpy()

    # Save the arrays as .npy files
    np.save("X_train.npy", X_train)
    np.save("X_val.npy", X_val)
    np.save("X_test.npy", X_test)
    np.save("y_train.npy", y_train)
    np.save("y_val.npy", y_val)
    np.save("y_test.npy", y_test)

    print("CSV onverted to .npy format.")


In [ ]:
to_npy(X_train_csv, X_val_csv, X_test_csv, y_train_csv, y_val_csv, y_test_csv)

CSV onverted to .npy format.


In [ ]:
train_list = ["X_train.npy", "X_val.npy", "X_test.npy", "y_train.npy", "y_val.npy", "y_test.npy"]

In [ ]:
def load(train_list):
    X_train = np.load(train_list[0])
    X_val = np.load(train_list[1])
    X_test = np.load(train_list[2])
    y_train = np.load(train_list[3])
    y_val = np.load(train_list[4])
    y_test = np.load(train_list[5])
    return X_train, X_val, X_test, y_train, y_val, y_test


def create_confusion_matrix(y_true, y_pred, class_labels):
    '''
    Description: Create a confusion matrix for multiclass classification.
    Arguments:
        y_true (array-like): Ground truth labels;
        y_pred (array-like): Predicted labels;
        class_labels (list): List of class names or indices.
    Outputs:
        pd.DataFrame: A confusion matrix as a pandas DataFrame.
    '''
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=[f"True {cls}" for cls in class_labels],
                         columns=[f"Predicted {cls}" for cls in class_labels])
    return cm_df


def score_metrics(y_true, y_pred, y_prob, average='weighted'):
    '''
    Description: Calculate various metrics for multiclass classification.
    Arguments:
        y_true (array-like): Ground truth labels;
        y_pred (array-like): Predicted labels;
        y_prob (array-like): Predicted probabilities for each class;
        average (str): Averaging method for metrics ('micro', 'macro', or 'weighted').
    Outputs:
        dict: A dictionary containing accuracy, ROC AUC, precision, recall, and F1.
    '''
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "ROC AUC": roc_auc_score(y_true, y_prob, multi_class="ovo", average=average),
        "Precision": precision_score(y_true, y_pred, average=average),
        "Recall": recall_score(y_true, y_pred, average=average),
        "F1 Score": f1_score(y_true, y_pred, average=average),
    }
    return metrics

def plot_curves(y_true, y_prob, class_labels):
    '''
    Description: Plot ROC and PR curves for multiclass classification.
    Arguments:
        y_true (array-like): Ground truth labels;
        y_prob (array-like): Predicted probabilities for each class;
        class_labels (list): List of class names or indices.
    Outputs: ROC and PR curves for each class.
    '''
    # ROC Curve for each class
    for i, class_label in enumerate(class_labels):
        RocCurveDisplay.from_predictions(
            y_true == i, y_prob[:, i], name=f"Class {class_label}"
        )
        plt.title(f"ROC Curve for Class {class_label}")
        plt.show()

    # PR Curve for each class
    for i, class_label in enumerate(class_labels):
        PrecisionRecallDisplay.from_predictions(
            y_true == i, y_prob[:, i], name=f"Class {class_label}"
        )
        plt.title(f"Precision-Recall Curve for Class {class_label}")
        plt.show()



def train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed, class_labels):
    """
    Trains and evaluates multiple models using train, validation, and test sets.
    Saves metrics, confusion matrices, ROC/PR curves, and a summary JSON file.

    Args:
        models (dict): Dictionary of models to evaluate.
        X_train, y_train: Training data.
        X_val, y_val: Validation data.
        X_test, y_test: Test data.
        seed (int): Random seed.
        class_labels (list): Class names.

    Returns:
        dict: Validation and test metrics for all models.
    """
    results = {}

    for model_name, model in tqdm(models.items(), desc="Training Models", unit="model"):
        print(f"\nTraining and evaluating: {model_name}")

        # Create storage directory
        model_dir = f"results/{model_name.replace(' ', '_')}"
        if os.path.exists(model_dir):
            shutil.rmtree(model_dir)
        os.makedirs(model_dir)

        # Train
        model.fit(X_train, y_train)

        # Save the model
        model_path = os.path.join(model_dir, "model.pkl")
        joblib.dump(model, model_path)

        # Validation
        y_val_pred = model.predict(X_val)
        y_val_prob = model.predict_proba(X_val) if hasattr(model, "predict_proba") else model.decision_function(X_val)
        val_acc = np.mean(y_val_pred == y_val)
        val_auc = roc_auc_score(y_val, y_val_prob, multi_class="ovr", average="weighted")
        val_mcc = matthews_corrcoef(y_val, y_val_pred)

        # Test
        y_test_pred = model.predict(X_test)
        y_test_prob = model.predict_proba(X_test) if hasattr(model, "predict_proba") else model.decision_function(X_test)
        test_acc = np.mean(y_test_pred == y_test)
        test_auc = roc_auc_score(y_test, y_test_prob, multi_class="ovr", average="weighted")
        test_mcc = matthews_corrcoef(y_test, y_test_pred)

        # Store metrics
        results[model_name] = {
            "Validation Accuracy": val_acc,
            "Validation ROC AUC": val_auc,
            "Validation MCC": val_mcc,
            "Test Accuracy": test_acc,
            "Test ROC AUC": test_auc,
            "Test MCC": test_mcc
        }

        # Save classification report
        report = classification_report(y_val, y_val_pred, target_names=class_labels)
        with open(os.path.join(model_dir, "classification_report.txt"), "w") as f:
            f.write(report)

        # Confusion matrix
        cm = confusion_matrix(y_val, y_val_pred)
        plt.figure(figsize=(6, 6))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title(f"Confusion Matrix - {model_name}")
        plt.colorbar()
        tick_marks = np.arange(len(class_labels))
        plt.xticks(tick_marks, class_labels)
        plt.yticks(tick_marks, class_labels)
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.savefig(os.path.join(model_dir, "confusion_matrix.png"))
        plt.close()

    summary_path = "results/summary_metrics.json"
    with open(summary_path, "w") as f:
        json.dump(results, f, indent=4)
    print(f"\nJSON saved to: {summary_path}")

    # Print final summary using MCC
    print("\nValidation and Test Metrics (using MCC):")
    for model_name, metrics in results.items():
        print(
            f"{model_name}: "
            f"Val Accuracy = {metrics['Validation Accuracy']:.4f}, "
            f"Val MCC = {metrics['Validation MCC']:.4f}, "
            f"Test Accuracy = {metrics['Test Accuracy']:.4f}, "
            f"Test MCC = {metrics['Test MCC']:.4f}"
        )

    return results

#AGORA-MC-I-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.787531 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 438111, number of used features: 424
[LightGBM] [Info] Start training from score -1.935235
[LightGBM] [Info] Start training from score -1.196584
[LightGBM] [Info] Start training from score -0.591702


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [01:15<02:31, 75.66s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:43:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [02:52<01:28, 88.16s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be rem


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.992317 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 438111, number of used features: 424
[LightGBM] [Info] Start training from score -1.935235
[LightGBM] [Info] Start training from score -1.196584
[LightGBM] [Info] Start training from score -0.591702


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [04:09<00:00, 83.18s/model]


✅ Summary JSON saved to: results/summary_metrics.json

📊 Summary of Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.4223, Val MCC = 0.7124, Test Accuracy = 0.4205, Test MCC = 0.7105
XGBoost: Val Accuracy = 0.4212, Val MCC = 0.7348, Test Accuracy = 0.4197, Test MCC = 0.7345
CatBoost: Val Accuracy = 0.4223, Val MCC = 0.7124, Test Accuracy = 0.4205, Test MCC = 0.7105


{'LightGBM': {'Validation Accuracy': np.float64(0.422262125207469),
  'Validation ROC AUC': np.float64(0.9531233421674931),
  'Validation MCC': np.float64(0.7124091678038387),
  'Test Accuracy': np.float64(0.4204807704633471),
  'Test ROC AUC': np.float64(0.9526018024882547),
  'Test MCC': np.float64(0.710457985584438)},
 'XGBoost': {'Validation Accuracy': np.float64(0.42116579935859216),
  'Validation ROC AUC': np.float64(0.9599320821922593),
  'Validation MCC': np.float64(0.7347784517450257),
  'Test Accuracy': np.float64(0.419664989739051),
  'Test ROC AUC': np.float64(0.9595731022550454),
  'Test MCC': np.float64(0.7345427810439527)},
 'CatBoost': {'Validation Accuracy': np.float64(0.422262125207469),
  'Validation ROC AUC': np.float64(0.9531233421674931),
  'Validation MCC': np.float64(0.7124091678038387),
  'Test Accuracy': np.float64(0.4204807704633471),
  'Test ROC AUC': np.float64(0.9526018024882547),
  'Test MCC': np.float64(0.710457985584438)}}

# AGORA-MC-II-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.822628 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 555496, number of used features: 424
[LightGBM] [Info] Start training from score -0.829590
[LightGBM] [Info] Start training from score -2.054973
[LightGBM] [Info] Start training from score -0.830858


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [02:07<04:14, 127.32s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:51:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [04:21<02:11, 131.27s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be re


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.671217 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 555496, number of used features: 424
[LightGBM] [Info] Start training from score -0.829590
[LightGBM] [Info] Start training from score -2.054973
[LightGBM] [Info] Start training from score -0.830858


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [06:15<00:00, 125.32s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.4246, Val MCC = 0.6833, Test Accuracy = 0.4242, Test MCC = 0.6843
XGBoost: Val Accuracy = 0.4220, Val MCC = 0.6988, Test Accuracy = 0.4214, Test MCC = 0.7006
CatBoost: Val Accuracy = 0.4246, Val MCC = 0.6833, Test Accuracy = 0.4242, Test MCC = 0.6843


{'LightGBM': {'Validation Accuracy': np.float64(0.42462687008389816),
  'Validation ROC AUC': np.float64(0.9289488521607994),
  'Validation MCC': np.float64(0.6832600387302309),
  'Test Accuracy': np.float64(0.4241737732743023),
  'Test ROC AUC': np.float64(0.9295957878396498),
  'Test MCC': np.float64(0.6843380570938078)},
 'XGBoost': {'Validation Accuracy': np.float64(0.4220370963881844),
  'Validation ROC AUC': np.float64(0.9381133554132581),
  'Validation MCC': np.float64(0.6987674581338895),
  'Test Accuracy': np.float64(0.42140171793538844),
  'Test ROC AUC': np.float64(0.9385859332596389),
  'Test MCC': np.float64(0.7005661100903448)},
 'CatBoost': {'Validation Accuracy': np.float64(0.42462687008389816),
  'Validation ROC AUC': np.float64(0.9289488521607994),
  'Validation MCC': np.float64(0.6832600387302309),
  'Test Accuracy': np.float64(0.4241737732743023),
  'Test ROC AUC': np.float64(0.9295957878396498),
  'Test MCC': np.float64(0.6843380570938078)}}

#AGORA-MC-III-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.436386 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 751295, number of used features: 424
[LightGBM] [Info] Start training from score -2.475510
[LightGBM] [Info] Start training from score -1.735178
[LightGBM] [Info] Start training from score -1.131358
[LightGBM] [Info] Start training from score -2.356179
[LightGBM] [Info] Start training from score -1.132786


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [03:25<06:50, 205.47s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:42:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [07:10<03:37, 217.22s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be re


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.443743 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 751295, number of used features: 424
[LightGBM] [Info] Start training from score -2.475510
[LightGBM] [Info] Start training from score -1.735178
[LightGBM] [Info] Start training from score -1.131358
[LightGBM] [Info] Start training from score -2.356179
[LightGBM] [Info] Start training from score -1.132786


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [10:21<00:00, 207.18s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.2843, Val MCC = 0.5280, Test Accuracy = 0.2845, Test MCC = 0.5295
XGBoost: Val Accuracy = 0.2805, Val MCC = 0.5475, Test Accuracy = 0.2809, Test MCC = 0.5473
CatBoost: Val Accuracy = 0.2843, Val MCC = 0.5280, Test Accuracy = 0.2845, Test MCC = 0.5295


{'LightGBM': {'Validation Accuracy': np.float64(0.2842633087831536),
  'Validation ROC AUC': np.float64(0.8833818392662312),
  'Validation MCC': np.float64(0.528027057806396),
  'Test Accuracy': np.float64(0.28450987286544877),
  'Test ROC AUC': np.float64(0.8837867765931178),
  'Test MCC': np.float64(0.5294668306686349)},
 'XGBoost': {'Validation Accuracy': np.float64(0.28046265311783),
  'Validation ROC AUC': np.float64(0.8927654293902281),
  'Validation MCC': np.float64(0.5474946051737586),
  'Test Accuracy': np.float64(0.28091498583330704),
  'Test ROC AUC': np.float64(0.8928773798904527),
  'Test MCC': np.float64(0.5472856313529633)},
 'CatBoost': {'Validation Accuracy': np.float64(0.2842633087831536),
  'Validation ROC AUC': np.float64(0.8833818392662312),
  'Validation MCC': np.float64(0.528027057806396),
  'Test Accuracy': np.float64(0.28450987286544877),
  'Test ROC AUC': np.float64(0.8837867765931178),
  'Test MCC': np.float64(0.5294668306686349)}}

#AGORA-MC-I-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.379751 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 846
[LightGBM] [Info] Number of data points in the train set: 338353, number of used features: 423
[LightGBM] [Info] Start training from score -3.520006
[LightGBM] [Info] Start training from score -2.041606
[LightGBM] [Info] Start training from score -0.173662


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:47<01:35, 47.97s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:07:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [01:47<00:54, 54.71s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be rem


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.444431 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 846
[LightGBM] [Info] Number of data points in the train set: 338353, number of used features: 423
[LightGBM] [Info] Start training from score -3.520006
[LightGBM] [Info] Start training from score -2.041606
[LightGBM] [Info] Start training from score -0.173662


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [02:37<00:00, 52.52s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.7776, Val MCC = 0.5028, Test Accuracy = 0.7778, Test MCC = 0.5085
XGBoost: Val Accuracy = 0.7736, Val MCC = 0.5153, Test Accuracy = 0.7734, Test MCC = 0.5208
CatBoost: Val Accuracy = 0.7775, Val MCC = 0.5007, Test Accuracy = 0.7779, Test MCC = 0.5087


{'LightGBM': {'Validation Accuracy': np.float64(0.7776032120403713),
  'Validation ROC AUC': np.float64(0.9305143145888296),
  'Validation MCC': np.float64(0.5027881838086422),
  'Test Accuracy': np.float64(0.7778486902501015),
  'Test ROC AUC': np.float64(0.931195242046915),
  'Test MCC': np.float64(0.5084683418112973)},
 'XGBoost': {'Validation Accuracy': np.float64(0.7735654494880254),
  'Validation ROC AUC': np.float64(0.9342456159582608),
  'Validation MCC': np.float64(0.515292142305787),
  'Test Accuracy': np.float64(0.7734482542114954),
  'Test ROC AUC': np.float64(0.9353588424914906),
  'Test MCC': np.float64(0.5208023162649784)},
 'CatBoost': {'Validation Accuracy': np.float64(0.7774996082586759),
  'Validation ROC AUC': np.float64(0.9302260505220822),
  'Validation MCC': np.float64(0.5007273626709302),
  'Test Accuracy': np.float64(0.7778788941196172),
  'Test ROC AUC': np.float64(0.9311360346958406),
  'Test MCC': np.float64(0.5086941683398922)}}

# AGORA-MC-II-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.666208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 644742, number of used features: 424
[LightGBM] [Info] Start training from score -0.818563
[LightGBM] [Info] Start training from score -2.147184
[LightGBM] [Info] Start training from score -0.816168


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [02:12<04:25, 132.85s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:20:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [04:33<02:17, 137.50s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be re


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.494641 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 644742, number of used features: 424
[LightGBM] [Info] Start training from score -0.818563
[LightGBM] [Info] Start training from score -2.147184
[LightGBM] [Info] Start training from score -0.816168


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [06:45<00:00, 135.31s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.4358, Val MCC = 0.4438, Test Accuracy = 0.4352, Test MCC = 0.4438
XGBoost: Val Accuracy = 0.4343, Val MCC = 0.4588, Test Accuracy = 0.4337, Test MCC = 0.4584
CatBoost: Val Accuracy = 0.4358, Val MCC = 0.4438, Test Accuracy = 0.4352, Test MCC = 0.4438


{'LightGBM': {'Validation Accuracy': np.float64(0.43579775562667117),
  'Validation ROC AUC': np.float64(0.8253162889995638),
  'Validation MCC': np.float64(0.44376233638924817),
  'Test Accuracy': np.float64(0.4351901461316956),
  'Test ROC AUC': np.float64(0.8253852234122853),
  'Test MCC': np.float64(0.44384662064234565)},
 'XGBoost': {'Validation Accuracy': np.float64(0.4342737818185562),
  'Validation ROC AUC': np.float64(0.834350203967905),
  'Validation MCC': np.float64(0.458758606512029),
  'Test Accuracy': np.float64(0.4337476512741263),
  'Test ROC AUC': np.float64(0.8345216041624915),
  'Test MCC': np.float64(0.4584290950505986)},
 'CatBoost': {'Validation Accuracy': np.float64(0.43579775562667117),
  'Validation ROC AUC': np.float64(0.8253162889995638),
  'Validation MCC': np.float64(0.44376233638924817),
  'Test Accuracy': np.float64(0.4351901461316956),
  'Test ROC AUC': np.float64(0.8253852234122853),
  'Test MCC': np.float64(0.44384662064234565)}}

#AGORA-MC-III-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.549162 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 698443, number of used features: 424
[LightGBM] [Info] Start training from score -4.239591
[LightGBM] [Info] Start training from score -2.776530
[LightGBM] [Info] Start training from score -0.897653
[LightGBM] [Info] Start training from score -2.223596
[LightGBM] [Info] Start training from score -0.897498


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [02:43<05:26, 163.20s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:29:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [05:46<02:55, 175.25s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be re


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.533100 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 848
[LightGBM] [Info] Number of data points in the train set: 698443, number of used features: 424
[LightGBM] [Info] Start training from score -4.239591
[LightGBM] [Info] Start training from score -2.776530
[LightGBM] [Info] Start training from score -0.897653
[LightGBM] [Info] Start training from score -2.223596
[LightGBM] [Info] Start training from score -0.897498


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [08:29<00:00, 169.70s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.3981, Val MCC = 0.3996, Test Accuracy = 0.3994, Test MCC = 0.4047
XGBoost: Val Accuracy = 0.3973, Val MCC = 0.4099, Test Accuracy = 0.3984, Test MCC = 0.4137
CatBoost: Val Accuracy = 0.3981, Val MCC = 0.3996, Test Accuracy = 0.3994, Test MCC = 0.4047


{'LightGBM': {'Validation Accuracy': np.float64(0.39811903441029367),
  'Validation ROC AUC': np.float64(0.8181034081933556),
  'Validation MCC': np.float64(0.39964167545130785),
  'Test Accuracy': np.float64(0.3994332285736168),
  'Test ROC AUC': np.float64(0.8197812012873344),
  'Test MCC': np.float64(0.4047089952155461)},
 'XGBoost': {'Validation Accuracy': np.float64(0.3972690101273393),
  'Validation ROC AUC': np.float64(0.824911461131207),
  'Validation MCC': np.float64(0.40992002034841507),
  'Test Accuracy': np.float64(0.3984324218301376),
  'Test ROC AUC': np.float64(0.8265205573439174),
  'Test MCC': np.float64(0.4136669888310119)},
 'CatBoost': {'Validation Accuracy': np.float64(0.39811903441029367),
  'Validation ROC AUC': np.float64(0.8181034081933556),
  'Validation MCC': np.float64(0.39964167545130785),
  'Test Accuracy': np.float64(0.3994332285736168),
  'Test ROC AUC': np.float64(0.8197812012873344),
  'Test MCC': np.float64(0.4047089952155461)}}

#CARVEME-MC-I-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.808094 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 363223, number of used features: 498
[LightGBM] [Info] Start training from score -1.078768
[LightGBM] [Info] Start training from score -1.557504
[LightGBM] [Info] Start training from score -0.800010


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [01:03<02:06, 63.45s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:15:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [02:19<01:10, 70.74s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be rem


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.772198 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 994
[LightGBM] [Info] Number of data points in the train set: 363223, number of used features: 497
[LightGBM] [Info] Start training from score -1.078768
[LightGBM] [Info] Start training from score -1.557504
[LightGBM] [Info] Start training from score -0.800010


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [03:18<00:00, 66.20s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.3688, Val MCC = 0.6190, Test Accuracy = 0.3687, Test MCC = 0.6153
XGBoost: Val Accuracy = 0.3675, Val MCC = 0.6422, Test Accuracy = 0.3674, Test MCC = 0.6405
CatBoost: Val Accuracy = 0.3688, Val MCC = 0.6190, Test Accuracy = 0.3687, Test MCC = 0.6153


{'LightGBM': {'Validation Accuracy': np.float64(0.3687686104362508),
  'Validation ROC AUC': np.float64(0.9155677042162186),
  'Validation MCC': np.float64(0.6190035116916786),
  'Test Accuracy': np.float64(0.36869397571458007),
  'Test ROC AUC': np.float64(0.9144688034214438),
  'Test MCC': np.float64(0.6152578528363983)},
 'XGBoost': {'Validation Accuracy': np.float64(0.3674807241839347),
  'Validation ROC AUC': np.float64(0.9246891527726856),
  'Validation MCC': np.float64(0.6422370026025572),
  'Test Accuracy': np.float64(0.367403833193155),
  'Test ROC AUC': np.float64(0.9237532924939472),
  'Test MCC': np.float64(0.6404750333949292)},
 'CatBoost': {'Validation Accuracy': np.float64(0.3687686104362508),
  'Validation ROC AUC': np.float64(0.9155677042162186),
  'Validation MCC': np.float64(0.6190035116916786),
  'Test Accuracy': np.float64(0.36869397571458007),
  'Test ROC AUC': np.float64(0.9144688034214438),
  'Test MCC': np.float64(0.6152578528363983)}}

#CARVEME-MC-II-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.868789 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 393053, number of used features: 498
[LightGBM] [Info] Start training from score -0.876869
[LightGBM] [Info] Start training from score -1.784365
[LightGBM] [Info] Start training from score -0.877040


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [01:06<02:13, 66.60s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:21:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [02:30<01:16, 76.85s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be rem


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.904723 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 393053, number of used features: 498
[LightGBM] [Info] Start training from score -0.876869
[LightGBM] [Info] Start training from score -1.784365
[LightGBM] [Info] Start training from score -0.877040


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [03:39<00:00, 73.03s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.4080, Val MCC = 0.5631, Test Accuracy = 0.4091, Test MCC = 0.5661
XGBoost: Val Accuracy = 0.4046, Val MCC = 0.5748, Test Accuracy = 0.4057, Test MCC = 0.5783
CatBoost: Val Accuracy = 0.4080, Val MCC = 0.5631, Test Accuracy = 0.4091, Test MCC = 0.5661


{'LightGBM': {'Validation Accuracy': np.float64(0.4080175692527074),
  'Validation ROC AUC': np.float64(0.8716059795903909),
  'Validation MCC': np.float64(0.5631351351862138),
  'Test Accuracy': np.float64(0.4091107073802299),
  'Test ROC AUC': np.float64(0.8714824580324824),
  'Test MCC': np.float64(0.5661187971041602)},
 'XGBoost': {'Validation Accuracy': np.float64(0.4046426596417987),
  'Validation ROC AUC': np.float64(0.8844749768252026),
  'Validation MCC': np.float64(0.5748492579077589),
  'Test Accuracy': np.float64(0.40566548051735124),
  'Test ROC AUC': np.float64(0.8840287319993244),
  'Test MCC': np.float64(0.5783074725945463)},
 'CatBoost': {'Validation Accuracy': np.float64(0.4080175692527074),
  'Validation ROC AUC': np.float64(0.8716059795903909),
  'Validation MCC': np.float64(0.5631351351862138),
  'Test Accuracy': np.float64(0.4091107073802299),
  'Test ROC AUC': np.float64(0.8714824580324824),
  'Test MCC': np.float64(0.5661187971041602)}}

#CARVEME-MC-III-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.780790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 998
[LightGBM] [Info] Number of data points in the train set: 592816, number of used features: 499
[LightGBM] [Info] Start training from score -1.570507
[LightGBM] [Info] Start training from score -2.046862
[LightGBM] [Info] Start training from score -1.288383
[LightGBM] [Info] Start training from score -2.195502
[LightGBM] [Info] Start training from score -1.287710


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [02:15<04:31, 135.64s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:54:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [04:50<02:27, 147.11s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be re


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.724819 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 998
[LightGBM] [Info] Number of data points in the train set: 592816, number of used features: 499
[LightGBM] [Info] Start training from score -1.570507
[LightGBM] [Info] Start training from score -2.046862
[LightGBM] [Info] Start training from score -1.288383
[LightGBM] [Info] Start training from score -2.195502
[LightGBM] [Info] Start training from score -1.287710


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [07:06<00:00, 142.03s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.2465, Val MCC = 0.4412, Test Accuracy = 0.2470, Test MCC = 0.4407
XGBoost: Val Accuracy = 0.2435, Val MCC = 0.4562, Test Accuracy = 0.2439, Test MCC = 0.4554
CatBoost: Val Accuracy = 0.2465, Val MCC = 0.4412, Test Accuracy = 0.2470, Test MCC = 0.4407


{'LightGBM': {'Validation Accuracy': np.float64(0.24649709217040325),
  'Validation ROC AUC': np.float64(0.8376789380955072),
  'Validation MCC': np.float64(0.44118115728276036),
  'Test Accuracy': np.float64(0.24698799065980478),
  'Test ROC AUC': np.float64(0.8373773874586743),
  'Test MCC': np.float64(0.44074448868195504)},
 'XGBoost': {'Validation Accuracy': np.float64(0.24349263544787036),
  'Validation ROC AUC': np.float64(0.8479713224932163),
  'Validation MCC': np.float64(0.45622396250735786),
  'Test Accuracy': np.float64(0.24386879086343377),
  'Test ROC AUC': np.float64(0.8475045929522653),
  'Test MCC': np.float64(0.45537891825671906)},
 'CatBoost': {'Validation Accuracy': np.float64(0.24649709217040325),
  'Validation ROC AUC': np.float64(0.8376789426470675),
  'Validation MCC': np.float64(0.44118115728276036),
  'Test Accuracy': np.float64(0.24698799065980478),
  'Test ROC AUC': np.float64(0.8373773843550429),
  'Test MCC': np.float64(0.44074448868195504)}}

#CARVEME-MC-I-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.488894 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 994
[LightGBM] [Info] Number of data points in the train set: 336413, number of used features: 497
[LightGBM] [Info] Start training from score -1.790277
[LightGBM] [Info] Start training from score -2.030520
[LightGBM] [Info] Start training from score -0.354080


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:50<01:40, 50.14s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:26:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [01:56<00:59, 59.97s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be rem


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.475420 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 336413, number of used features: 498
[LightGBM] [Info] Start training from score -1.790277
[LightGBM] [Info] Start training from score -2.030520
[LightGBM] [Info] Start training from score -0.354080


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [02:45<00:00, 55.27s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.5891, Val MCC = 0.4568, Test Accuracy = 0.5875, Test MCC = 0.4609
XGBoost: Val Accuracy = 0.5857, Val MCC = 0.4699, Test Accuracy = 0.5842, Test MCC = 0.4703
CatBoost: Val Accuracy = 0.5891, Val MCC = 0.4568, Test Accuracy = 0.5875, Test MCC = 0.4609


{'LightGBM': {'Validation Accuracy': np.float64(0.5890557306502555),
  'Validation ROC AUC': np.float64(0.8976190295048228),
  'Validation MCC': np.float64(0.4567799624252035),
  'Test Accuracy': np.float64(0.587517984961445),
  'Test ROC AUC': np.float64(0.8965359286280593),
  'Test MCC': np.float64(0.46085193862441914)},
 'XGBoost': {'Validation Accuracy': np.float64(0.5856954579098207),
  'Validation ROC AUC': np.float64(0.9022626210106085),
  'Validation MCC': np.float64(0.46987173200491456),
  'Test Accuracy': np.float64(0.5841782569062333),
  'Test ROC AUC': np.float64(0.9008387346903566),
  'Test MCC': np.float64(0.47025776286017573)},
 'CatBoost': {'Validation Accuracy': np.float64(0.5890557306502555),
  'Validation ROC AUC': np.float64(0.8976190295048228),
  'Validation MCC': np.float64(0.4567799624252035),
  'Test Accuracy': np.float64(0.587517984961445),
  'Test ROC AUC': np.float64(0.8965359286280593),
  'Test MCC': np.float64(0.46085193862441914)}}

#CARVEME-MC-II-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.848705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 553606, number of used features: 498
[LightGBM] [Info] Start training from score -0.853423
[LightGBM] [Info] Start training from score -1.905548
[LightGBM] [Info] Start training from score -0.854950


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [02:15<04:30, 135.12s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:33:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [04:28<02:14, 134.23s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be re


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.062337 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 553606, number of used features: 498
[LightGBM] [Info] Start training from score -0.853423
[LightGBM] [Info] Start training from score -1.905548
[LightGBM] [Info] Start training from score -0.854950


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [06:23<00:00, 127.70s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.4249, Val MCC = 0.3849, Test Accuracy = 0.4244, Test MCC = 0.3846
XGBoost: Val Accuracy = 0.4243, Val MCC = 0.3921, Test Accuracy = 0.4239, Test MCC = 0.3927
CatBoost: Val Accuracy = 0.4249, Val MCC = 0.3849, Test Accuracy = 0.4244, Test MCC = 0.3846


{'LightGBM': {'Validation Accuracy': np.float64(0.42486448291609413),
  'Validation ROC AUC': np.float64(0.7861349346244991),
  'Validation MCC': np.float64(0.3849032915803742),
  'Test Accuracy': np.float64(0.4243962999840896),
  'Test ROC AUC': np.float64(0.7860081372028446),
  'Test MCC': np.float64(0.3845645334592783)},
 'XGBoost': {'Validation Accuracy': np.float64(0.4242629962915202),
  'Validation ROC AUC': np.float64(0.7924229122865704),
  'Validation MCC': np.float64(0.39206416390695326),
  'Test Accuracy': np.float64(0.42387210744887005),
  'Test ROC AUC': np.float64(0.7927350980539624),
  'Test MCC': np.float64(0.39267657829985814)},
 'CatBoost': {'Validation Accuracy': np.float64(0.42486448291609413),
  'Validation ROC AUC': np.float64(0.7861349346244991),
  'Validation MCC': np.float64(0.3849032915803742),
  'Test Accuracy': np.float64(0.4243962999840896),
  'Test ROC AUC': np.float64(0.7860081372028446),
  'Test MCC': np.float64(0.3845645334592783)}}


#CARVEME-MC-III-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.729776 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 998
[LightGBM] [Info] Number of data points in the train set: 592816, number of used features: 499
[LightGBM] [Info] Start training from score -1.570507
[LightGBM] [Info] Start training from score -2.046862
[LightGBM] [Info] Start training from score -1.288383
[LightGBM] [Info] Start training from score -2.195502
[LightGBM] [Info] Start training from score -1.287710


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [02:16<04:32, 136.41s/model]


Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:04:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [04:48<02:25, 145.83s/model]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be re


Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.882402 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 998
[LightGBM] [Info] Number of data points in the train set: 592816, number of used features: 499
[LightGBM] [Info] Start training from score -1.570507
[LightGBM] [Info] Start training from score -2.046862
[LightGBM] [Info] Start training from score -1.288383
[LightGBM] [Info] Start training from score -2.195502
[LightGBM] [Info] Start training from score -1.287710


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [07:05<00:00, 141.93s/model]


JSON saved to: results/summary_metrics.json

Validation and Test Metrics (using MCC):
LightGBM: Val Accuracy = 0.2465, Val MCC = 0.4412, Test Accuracy = 0.2470, Test MCC = 0.4407
XGBoost: Val Accuracy = 0.2435, Val MCC = 0.4562, Test Accuracy = 0.2439, Test MCC = 0.4554
CatBoost: Val Accuracy = 0.2465, Val MCC = 0.4412, Test Accuracy = 0.2470, Test MCC = 0.4407


{'LightGBM': {'Validation Accuracy': np.float64(0.24649709217040325),
  'Validation ROC AUC': np.float64(0.8376789380955072),
  'Validation MCC': np.float64(0.44118115728276036),
  'Test Accuracy': np.float64(0.24698799065980478),
  'Test ROC AUC': np.float64(0.8373773874586743),
  'Test MCC': np.float64(0.44074448868195504)},
 'XGBoost': {'Validation Accuracy': np.float64(0.24349263544787036),
  'Validation ROC AUC': np.float64(0.8479713224932163),
  'Validation MCC': np.float64(0.45622396250735786),
  'Test Accuracy': np.float64(0.24386879086343377),
  'Test ROC AUC': np.float64(0.8475045929522653),
  'Test MCC': np.float64(0.45537891825671906)},
 'CatBoost': {'Validation Accuracy': np.float64(0.24649709217040325),
  'Validation ROC AUC': np.float64(0.8376789426470675),
  'Validation MCC': np.float64(0.44118115728276036),
  'Test Accuracy': np.float64(0.24698799065980478),
  'Test ROC AUC': np.float64(0.8373773843550429),
  'Test MCC': np.float64(0.44074448868195504)}}

#CARVEME-TL-II-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Number of positive: 152771, number of negative: 119972
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.151010 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 272743, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.560128 -> initscore=0.241682
[LightGBM] [Info] Start training from score 0.241682
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Validation Classification Report for LightGBM:

              precision    recall  f1-score   support

           0       0.65      0.45      0.53     39516
           1       0.66      0.81      0.73     51398

    accuracy                           0.65     90914
   macro avg       0.65      0.63      0.63     90914
weighted avg       0.65      0.65      0.64     90914

Confusion Matrix for LightGBM (Validation):
 [[17917 21599]
 [ 9837 41561]]


Training Models:  33%|███▎      | 1/3 [00:18<00:37, 18.71s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 152771, number of negative: 119972
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.148366 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 272743, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.560128 -> initscore=0.241682
[LightGBM] [Info] Start training from score 0.241682
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:55<00:00, 18.57s/model]

Validation Classification Report for CatBoost:

              precision    recall  f1-score   support

           0       0.65      0.45      0.53     39516
           1       0.66      0.81      0.73     51398

    accuracy                           0.65     90914
   macro avg       0.65      0.63      0.63     90914
weighted avg       0.65      0.65      0.64     90914

Confusion Matrix for CatBoost (Validation):
 [[17917 21599]
 [ 9837 41561]]

Summary of Test Metrics:
LightGBM: Test Accuracy = 0.5236, Test Precision = 0.6565, Test Recall = 0.8064, Test F1 = 0.7238, Test AUC = 0.7122, Test MCC = 0.2846
XGBoost: Test Accuracy = 0.5177, Test Precision = 0.6820, Test Recall = 0.7808, Test F1 = 0.7281, Test AUC = 0.7379, Test MCC = 0.3251
CatBoost: Test Accuracy = 0.5236, Test Precision = 0.6565, Test Recall = 0.8064, Test F1 = 0.7238, Test AUC = 0.7122, Test MCC = 0.2846

Summary of Validation and Test Metrics:
LightGBM: Test Accuracy = 0.5236, Test MCC = 0.2846
XGBoost: Test Accuracy 

{'LightGBM': {'Test Accuracy': np.float64(0.5235509014358107),
  'Test Precision': 0.656478574729495,
  'Test Recall': 0.8064206714299696,
  'Test F1': 0.7237653507616086,
  'Test AUC': np.float64(0.7122044749694865),
  'Test MCC': np.float64(0.28455678837058573)},
 'XGBoost': {'Test Accuracy': np.float64(0.5177447065456777),
  'Test Precision': 0.6819914855784849,
  'Test Recall': 0.7808358617989625,
  'Test F1': 0.7280741781809553,
  'Test AUC': np.float64(0.7378698634312791),
  'Test MCC': np.float64(0.32508117909461914)},
 'CatBoost': {'Test Accuracy': np.float64(0.5235509014358107),
  'Test Precision': 0.656478574729495,
  'Test Recall': 0.8064206714299696,
  'Test F1': 0.7237653507616086,
  'Test AUC': np.float64(0.7122044749694865),
  'Test MCC': np.float64(0.28455678837058573)}}

#CARVEME-TL-II-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.406182 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 1109103, number of used features: 153
[LightGBM] [Info] Start training from score -0.840895
[LightGBM] [Info] Start training from score -1.982144
[LightGBM] [Info] Start training from score -0.841874


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#AGORA-TL-II-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]


Training and evaluating: LightGBM


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 88034, number of negative: 66418
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073693 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 154452, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.569976 -> initscore=0.281755
[LightGBM] [Info] Start training from score 0.281755
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:06<00:12,  6.31s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 88034, number of negative: 66418
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.068105 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 154452, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.569976 -> initscore=0.281755
[LightGBM] [Info] Start training from score 0.281755
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:18<00:00,  6.31s/model]

Validation Classification Report for CatBoost:

              precision    recall  f1-score   support

           0       0.62      0.69      0.66     22141
           1       0.75      0.68      0.71     29343

    accuracy                           0.69     51484
   macro avg       0.69      0.69      0.69     51484
weighted avg       0.69      0.69      0.69     51484

Confusion Matrix for CatBoost (Validation):
 [[15380  6761]
 [ 9271 20072]]

Summary of Test Metrics:
LightGBM: Test Accuracy = 0.5033, Test Precision = 0.7502, Test Recall = 0.6871, Test F1 = 0.7173, Test AUC = 0.7547, Test MCC = 0.3786
XGBoost: Test Accuracy = 0.5054, Test Precision = 0.7488, Test Recall = 0.7046, Test F1 = 0.7260, Test AUC = 0.7571, Test MCC = 0.3865
CatBoost: Test Accuracy = 0.5033, Test Precision = 0.7502, Test Recall = 0.6871, Test F1 = 0.7173, Test AUC = 0.7547, Test MCC = 0.3786

Summary of Validation and Test Metrics:
LightGBM: Test Accuracy = 0.5033, Test MCC = 0.3786
XGBoost: Test Accuracy 

{'LightGBM': {'Test Accuracy': np.float64(0.5033422713573665),
  'Test Precision': 0.750241223187115,
  'Test Recall': 0.6871282417320961,
  'Test F1': 0.7172991289229513,
  'Test AUC': np.float64(0.7547413667381978),
  'Test MCC': np.float64(0.3785846147389124)},
 'XGBoost': {'Test Accuracy': np.float64(0.5053964912946184),
  'Test Precision': 0.7487900021671603,
  'Test Recall': 0.7046327453179702,
  'Test F1': 0.7260405904705203,
  'Test AUC': np.float64(0.7571044946340006),
  'Test MCC': np.float64(0.3865054296911203)},
 'CatBoost': {'Test Accuracy': np.float64(0.5033422713573665),
  'Test Precision': 0.750241223187115,
  'Test Recall': 0.6871282417320961,
  'Test F1': 0.7172991289229513,
  'Test AUC': np.float64(0.7547413667381978),
  'Test MCC': np.float64(0.3785846147389124)}}

#AGORA-TL-II-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMClassifier(random_state=4221),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=4221),
    "CatBoost": LGBMClassifier(random_state=42)
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221, class_labels=['Comp', 'FCoop', 'O_plusx_Coop', 'O_xplus_Coop', 'O_xx_Coop'])

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Training and evaluating: LightGBM
[LightGBM] [Info] Number of positive: 208987, number of negative: 186574
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.244365 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 395561, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.528331 -> initscore=0.113444
[LightGBM] [Info] Start training from score 0.113444
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:37<01:14, 37.49s/model]

Validation Classification Report for LightGBM:

              precision    recall  f1-score   support

           0       0.68      0.64      0.66     62306
           1       0.69      0.73      0.71     69548

    accuracy                           0.69    131854
   macro avg       0.69      0.68      0.69    131854
weighted avg       0.69      0.69      0.69    131854

Confusion Matrix for LightGBM (Validation):
 [[39609 22697]
 [18501 51047]]

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:38:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


...

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 208987, number of negative: 186574
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.204433 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 395561, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.528331 -> initscore=0.113444
[LightGBM] [Info] Start training from score 0.113444
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [01:46<00:00, 35.46s/model]

Validation Classification Report for CatBoost:

              precision    recall  f1-score   support

           0       0.68      0.64      0.66     62306
           1       0.69      0.73      0.71     69548

    accuracy                           0.69    131854
   macro avg       0.69      0.68      0.69    131854
weighted avg       0.69      0.69      0.69    131854

Confusion Matrix for CatBoost (Validation):
 [[39609 22697]
 [18501 51047]]

Summary of Test Metrics:
LightGBM: Test Accuracy = 0.5035, Test Precision = 0.6899, Test Recall = 0.7335, Test F1 = 0.7110, Test AUC = 0.7496, Test MCC = 0.3662
XGBoost: Test Accuracy = 0.5031, Test Precision = 0.7089, Test Recall = 0.7443, Test F1 = 0.7262, Test AUC = 0.7726, Test MCC = 0.4036
CatBoost: Test Accuracy = 0.5035, Test Precision = 0.6899, Test Recall = 0.7335, Test F1 = 0.7110, Test AUC = 0.7496, Test MCC = 0.3662

Summary of Validation and Test Metrics:
LightGBM: Test Accuracy = 0.5035, Test MCC = 0.3662
XGBoost: Test Accuracy 

{'LightGBM': {'Test Accuracy': np.float64(0.5035070735701853),
  'Test Precision': 0.6898506898506899,
  'Test Recall': 0.7334682570440224,
  'Test F1': 0.7109911439165734,
  'Test AUC': np.float64(0.7496046431793731),
  'Test MCC': np.float64(0.3662426089883588)},
 'XGBoost': {'Test Accuracy': np.float64(0.5031101763280458),
  'Test Precision': 0.7088801399825022,
  'Test Recall': 0.7443195682441258,
  'Test F1': 0.7261677204332696,
  'Test AUC': np.float64(0.7725742211024463),
  'Test MCC': np.float64(0.4036409310070837)},
 'CatBoost': {'Test Accuracy': np.float64(0.5035070735701853),
  'Test Precision': 0.6898506898506899,
  'Test Recall': 0.7334682570440224,
  'Test F1': 0.7109911439165734,
  'Test AUC': np.float64(0.7496046431793731),
  'Test MCC': np.float64(0.3662426089883588)}}